# Markdown cells as docstrings

An opinion that emerged from `importnb` allows an end user to write their docstrings in rich Markdown syntax. This notebook illustrates how to acheive this using the `ast` representation of the module.

In [14]:
    from . import __Importing_notebooks_with_proper_source; import ast

A `SourceFileLoader` for notebooks that include leading strings to functions and classes as docstrings. We user the source_to_code method to acheive these results by modifying the ast representation of the module.

In [15]:
    class DocStringLoader(__Importing_notebooks_with_proper_source.FlatSourceLoader):
        def source_to_code(self, data, path, *, _optimize=-1):
            return super().source_to_code(update_docstring(ast.parse(data)), path, _optimize=_optimize)

## Transpose appropriate docstrings.

These helpers functions takes string expressions leading functions as the docstring.  The first cell is the docstring for the module.

In [25]:
    def update_docstring(module):
        from functools import reduce
        module.body = reduce(markdown_docstring, module.body, [])
        return module

`markdown_docstring` collects string expressions leading function and class defintions

In [26]:
    def markdown_docstring(nodes, node):
        if (len(nodes) > 1 and str_expr(nodes[-1]) 
            and isinstance(node, (ast.ClassDef, ast.FunctionDef)) 
            and not str_expr(node.body[0])): node.body.insert(0, nodes.pop())
        return nodes.append(node) or nodes


`str_expr` assesses if a node is a string expression.

In [27]:
    def str_expr(node):  return isinstance(node, ast.Expr) and isinstance(node.value, ast.Str)

## Demonstration

In [28]:
    from importlib import import_module; from IPython import get_ipython

In [29]:
    def _test_cells_as_docstrings():
        __Importing_notebooks_with_proper_source.__The_simplest_path_hook_importer_for_a_notebook.load_ipython_extension(get_ipython(), loader = DocStringLoader)
        module = import_module('deathbeds.2018-07-30-Markdown-cells-as-docstrings')
        assert __import__('inspect').getdoc(module)
        assert __import__('inspect').getdoc(module.str_expr)

### The HTML pager

Docstrings written as markdown have very nice representation when the __html_pager__ is enabled.

In [30]:
    def _test_the_pager():
        get_ipython().enable_html_pager = True
        module?